In [22]:
import pandas as pd
import folium
import numpy as np
import json
import requests

In [3]:
dfm = pd.read_csv("../resource/city_map/city_map.csv")

In [4]:
dfm.rename(columns={'avg(la)':'la','avg(AQI)':'AQI','avg(lo)':'lo'},inplace=True)

In [5]:
dfm.head(3)

,city,year,month,day,la,AQI,lo
0,北京,2018,8,13,38.0,57.289474,115.0
1,天津,2018,8,6,39.0,54.778846,117.0
2,青岛,2019,2,20,36.0,191.666667,119.0


In [6]:
latitude = 38.88
longitude = 115.49

In [7]:
a = {'北京':'北京市','厦门':'福建省','成都':'四川省','乌鲁木齐':'新疆维吾尔自治区','天津':'天津市','呼和浩特':'内蒙古自治区','昆明':'云南省','上海':'上海市','济南':'山东省','郑州':'河南省','海口':'海南省'}

In [8]:
set(dfm.city.values)

{'上海', '乌鲁木齐', '北京', '厦门', '呼和浩特', '天津', '成都', '昆明', '济南', '海口', '郑州', '青岛'}

In [9]:
df_day = dfm[(dfm.year==2019)&(dfm.month==3)&(dfm.day==30)]
df_day2 = df_day[df_day.city!='青岛'].copy()

In [290]:
df_day

,city,year,month,day,la,AQI,lo


In [10]:
df_2019_1_27 = dfm[(dfm.year==2019)&(dfm.month==1)&(dfm.day==27)]

In [11]:
se = df_2019_1_27.groupby(df_2019_1_27.city)['AQI'].agg(np.mean)

In [12]:
se.index

Index(['上海', '乌鲁木齐', '北京', '厦门', '呼和浩特', '天津', '成都', '昆明', '济南', '海口', '郑州',
       '青岛'],
      dtype='object', name='city')

In [13]:
import numpy as np
se_2019_1_27 = df_2019_1_27.groupby(df_2019_1_27.city)['AQI'].agg(np.mean)
data = {'city':list(se_2019_1_27.index),
       'AQI':list(se_2019_1_27.values)}
df_2019_1_27 = pd.DataFrame(data)
df_2019_1_27 = df_2019_1_27[df_2019_1_27.city!='青岛'].copy()
df_2019_1_27

,city,AQI
0,上海,46.078881
1,乌鲁木齐,146.977705
2,北京,129.850067
3,厦门,50.485317
4,呼和浩特,81.854749
5,天津,109.896637
6,成都,91.033679
7,昆明,41.808055
8,济南,97.081621
9,海口,51.715476


In [14]:
a.keys()

dict_keys(['北京', '厦门', '成都', '乌鲁木齐', '天津', '呼和浩特', '昆明', '上海', '济南', '郑州', '海口'])

In [15]:
for i in range(0,len(df_2019_1_27)):
    c = a.get(df_2019_1_27.loc[i].city)
    print(c)
    df_2019_1_27.loc[i,'city'] = a.get(df_2019_1_27.loc[i].city)

上海市
新疆维吾尔自治区
北京市
福建省
内蒙古自治区
天津市
四川省
云南省
山东省
海南省
河南省


In [16]:
df_city = df_2019_1_27.copy()
from branca.colormap import linear

colormap = linear.OrRd_05.scale(
    df_city.AQI.min(), df_city.AQI.max()
)

print(colormap(5.0))

colormap

#fef0d9ff


In [17]:
df_city_dict = df_city.set_index("city")["AQI"]
df_city_dict = df_city_dict.append(pd.Series([0],index=['None']))
df_city_dict

上海市          46.078881
新疆维吾尔自治区    146.977705
北京市         129.850067
福建省          50.485317
内蒙古自治区       81.854749
天津市         109.896637
四川省          91.033679
云南省          41.808055
山东省          97.081621
海南省          51.715476
河南省         121.001206
None          0.000000
dtype: float64

In [18]:
df_city_dict.index

Index(['上海市', '新疆维吾尔自治区', '北京市', '福建省', '内蒙古自治区', '天津市', '四川省', '云南省', '山东省',
       '海南省', '河南省', 'None'],
      dtype='object')

In [23]:
url = ('https://geo.datav.aliyun.com/areas_v2/bound/100000_full.json')
san_geo = f'{url}'
geo_json_data = json.loads(requests.get(san_geo).text)

In [24]:
m = folium.Map([latitude, longitude], zoom_start=4)

folium.GeoJson(
    geo_json_data,
    name="AQI",
    style_function=lambda feature: {
        "fillColor": colormap(df_city_dict[feature["properties"]["name"]]) if feature["properties"]["name"] in a.values()
        else "#f0f0f0",#colormap(df_city_dict["None"]),
        "color": "black",
        "weight": 1,
        "dashArray": "5, 5",
        "fillOpacity": 0.9,
    },
).add_to(m)

folium.LayerControl().add_to(m)

display(m)

In [25]:
from branca.colormap import linear
import folium
#constant
latitude = 38.88
longitude = 115.49
a = {'北京':'北京市','厦门':'福建省','成都':'四川省','乌鲁木齐':'新疆维吾尔自治区','天津':'天津市','呼和浩特':'内蒙古自治区','昆明':'云南省','上海':'上海市','济南':'山东省','郑州':'河南省','海口':'海南省'}
def translate(df):
    for i in range(0,len(df)):
        c = a.get(df.loc[i].city)
        df.loc[i,'city'] = a.get(df.loc[i].city)
    return df

def plotMap(dfm,year,month,day):
    df_day = dfm[(dfm.year==year)&(dfm.month==month)&(dfm.day==day)]
    if len(df_day)==0:
        print("there is no data of cities!!!")
        return 0
    df_day = df_day[df_day.city!='青岛'].copy()
    se_day = df_day.groupby(df_day.city)['AQI'].agg(np.mean)
    data = {'city':list(se_day.index),
       'AQI':list(se_day.values)}
    df_day = pd.DataFrame(data)
    df_day = translate(df_day)

    colormap = linear.OrRd_05.scale(
        df_day.AQI.min(), df_day.AQI.max()
    )

    df_city_dict = df_day.set_index("city")["AQI"]
    df_city_dict = df_city_dict.append(pd.Series([0],index=['None']))

    url = ('https://geo.datav.aliyun.com/areas_v2/bound/100000_full.json')
    san_geo = f'{url}'
    geo_json_data = json.loads(requests.get(san_geo).text)
    
    m = folium.Map([latitude, longitude], zoom_start=4)

    folium.GeoJson(
        geo_json_data,
        name="AQI",
        style_function=lambda feature: {
            "fillColor": colormap(df_city_dict[feature["properties"]["name"]]) if feature["properties"]["name"] in df_city_dict.index
            else "#f0f0f0",#colormap(df_city_dict["None"]),
            "color": "black",
            "weight": 1,
            "dashArray": "5, 5",
            "fillOpacity": 0.9,
        },
    ).add_to(m)

    folium.LayerControl().add_to(m)

    return m
    
    

In [26]:

display(plotMap(dfm,2018,8,25))
